In [1]:
import os

import torch
from  torch.cuda.amp import autocast
from PIL import Image
# from diffusers_interpret import StableDiffusionImg2ImgPipelineExplainer
from PIL import Image
from diffusers import StableDiffusionImg2ImgPipeline, DPMSolverMultistepScheduler
# from lavis.models import load_model_and_preprocess
from dotenv import find_dotenv, load_dotenv
from numba import cuda
from tqdm import tqdm as tqdm_notebook

In [2]:
load_dotenv(find_dotenv())

True

In [9]:
BASE_STRENGTH = 0.7
BASE_SCALE = 9
# MODEL_NAME = "CompVis/stable-diffusion-v1-4"
MODEL_NAME = "stabilityai/stable-diffusion-2-1-base"
IMG_SIZE = (512, 512)
SEED = 42
IMG_DEVICE = 'cuda'
# IMG_DEVICE = 'gpu'
DESC_DEVICE = 'cpu'
STYLES_LST = [
    "painting in the style of Banksy of",
    "painting in the style of Robert Delaunay of",
    "painting in the style of Vincent Van Gogh of",
    "painting in the style of Malevich of"
]

In [4]:
assert torch.cuda.is_available()
torch.cuda.empty_cache()

In [5]:
scheduler = DPMSolverMultistepScheduler.from_pretrained(MODEL_NAME, subfolder="scheduler")
pipe = StableDiffusionImg2ImgPipeline.from_pretrained(
    MODEL_NAME,
    use_auth_token=os.environ["DIFFUSERS_ACCESS_TOKEN"],
    scheduler=scheduler,
    safety_checker=None,
    torch_dtype=torch.float16,
    revision="fp16",
)
pipe.to(IMG_DEVICE)
pipe.enable_attention_slicing()
# pipe.enable_vae_slicing()
# pipe.enable_xformers_memory_efficient_attention()
pipe.enable_sequential_cpu_offload()

Fetching 12 files:   0%|          | 0/12 [00:00<?, ?it/s]

In [21]:
image = Image.open("../input_8eea7506-cd11-440f-b09e-49d6e33d844b.png").convert("RGB").resize(IMG_SIZE)
# prompt = "painting in the style of Banksy of"
prompt = "painting of a person face portrait"

In [31]:
torch.cuda.empty_cache()

with autocast(), torch.inference_mode():
    output = pipe(
        prompt=prompt, 
        image=image, 
        strength=0.25,
        # negative_prompt=negative_prompt,
        num_images_per_prompt=4,
        num_inference_steps=100,
        guidance_scale=10,
        generator=None,
    ).images

  0%|          | 0/25 [00:00<?, ?it/s]

In [28]:
torch.cuda.empty_cache()